In [0]:
!unzip -uq '/content/drive/My Drive/dataset.zip' -d '/content/Mask_Dataset'

In [0]:
import os
base_dir = '/content/Mask_Dataset/dataset'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test')
# Directory with our training mask pictures
train_mask_dir = os.path.join(train_dir, 'with_mask')

# Directory with our training without mask pictures
train_wmask_dir = os.path.join(train_dir, 'without_mask')

# Directory with our validation mask pictures
validation_mask_dir = os.path.join(validation_dir, 'with_mask_test')

# Directory with our validation without mask pictures
validation_wmask_dir = os.path.join(validation_dir, 'without_mask_test')

In [0]:
import tensorflow as tf
import keras_preprocessing
from tensorflow import keras
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

In [7]:

training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')


validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	train_dir,
	target_size=(300,300),
	class_mode='binary',
  batch_size=25
)

validation_generator = validation_datagen.flow_from_directory(
	validation_dir,
	target_size=(300,300),
	class_mode='binary',
  batch_size=15
)

Found 1292 images belonging to 2 classes.
Found 90 images belonging to 2 classes.


In [0]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [0]:
model.compile(optimizer = RMSprop(lr=1e-4), loss = 'binary_crossentropy' , metrics = ['accuracy'])

In [10]:
history = model.fit(train_generator, epochs=10, steps_per_epoch=25, validation_data = validation_generator, verbose = 1, validation_steps=6)

Epoch 1/10
25/25 [==============================] - 260s 10s/step - loss: 0.6858 - accuracy: 0.5840 - val_loss: 0.4330 - val_accuracy: 0.9111
Epoch 2/10
25/25 [==============================] - 257s 10s/step - loss: 0.4391 - accuracy: 0.8192 - val_loss: 0.2469 - val_accuracy: 0.9111
Epoch 3/10
25/25 [==============================] - 257s 10s/step - loss: 0.3538 - accuracy: 0.8914 - val_loss: 0.1464 - val_accuracy: 0.9667
Epoch 4/10
25/25 [==============================] - 257s 10s/step - loss: 0.2857 - accuracy: 0.8960 - val_loss: 0.1194 - val_accuracy: 0.9667
Epoch 5/10
25/25 [==============================] - 261s 10s/step - loss: 0.2892 - accuracy: 0.8880 - val_loss: 0.4701 - val_accuracy: 0.7444
Epoch 6/10
25/25 [==============================] - 254s 10s/step - loss: 0.2526 - accuracy: 0.9060 - val_loss: 0.0871 - val_accuracy: 0.9778
Epoch 7/10
25/25 [==============================] - 259s 10s/step - loss: 0.2457 - accuracy: 0.9173 - val_loss: 0.0847 - val_accuracy: 0.9778
Epoch 

In [24]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images,verbose=1, batch_size=10)
  print(fn)
  print(classes)
  if classes>[0.5]:
    print(fn , "is not wearing a mask")
  else:
    print(fn , "is wearing a mask")

Saving JoaquinPhoenix_test.jpg to JoaquinPhoenix_test.jpg
1/1 [==============================] - 0s 1ms/step
JoaquinPhoenix_test.jpg
[[1.]]
JoaquinPhoenix_test.jpg is not wearing a mask


In [22]:
model.save("mask_detection")

INFO:tensorflow:Assets written to: mask_detection/assets
